My goal here was to create a scalable bond amortization within Python. After creating this in excel, it became clear that one of the limitations of doing this in excel was the lack of a way to use a loop, making it nearly impossible (or at the very least impractical) to do this in excel.

At present, the output format is such so that it can be plugged into my MBS cashflow waterfall model in excel easily.s



In [ ]:
import pandas as pd
import numpy as np
#!pip install mortgage.  I can pip install right in the jupyter notebook
#import mortgage
import numpy_financial as npf
import sys
import os

file_path ="C:/Users/syrac/OneDrive/Desktop/Raymond James Case Study/Analyst Challenge - unedited file.xlsx"






In [ ]:
#The sys.path adds the following file path to the list of directories that import will look through
sys.path.append("C:/Users/syrac/OneDrive/Desktop/Raymond James Case Study")
import Amortizationtables

In [ ]:
df = pd.read_excel(file_path)

# Dictionary to store lists for each Mortgage
bond_data = {}
#List to store all amortization dataframes
amortization_tables = []
export_file = "C:/Users/syrac/OneDrive/Desktop/Raymond James Case Study/ Python Amortization output combined.xlsx"

# Iterate over the rows in the DataFrame. The iterrows method works on pairs of index and panda object "row"
for index, row in df.iterrows():
    # Get the ID (mortgage name) 
    bond_id = row['ID']
    
    # The .tolist method converts the the Pandas series into a regular python list from a pandas row. The drop method removes the ID column
    bond_values = row.drop('ID').tolist()
    
    # Creates a list for each ID 

    bond_data[bond_id] = bond_values #bond ID is the "key", bond values is the "Value pair", which is a list containing the bond info

    bond_values.append(12)
    
    cutoff_Date_Balance, Gross_Coupon, Remaining_Amortization, payments_per_year = bond_values
    Remaining_Amortization = Remaining_Amortization/12
    Gross_Coupon = Gross_Coupon/100
    amortization_df = Amortizationtables.create_amortization_table(cutoff_Date_Balance, Gross_Coupon, Remaining_Amortization, payments_per_year)
    amortization_df['Bond_ID'] = bond_id
    amortization_df['Period'] = range(1, len(amortization_df) + 1)
    amortization_tables.append(amortization_df)
combined_df = pd.concat(amortization_tables, ignore_index=True)   

total_payments = combined_df.groupby('Period')['Payment'].sum().reset_index() #The reset index is what creates the new dataframe here
total_payments.rename(columns={'Payment':'Total Payment', 'Period':'Unique Period'}, inplace = True)    #The merge is what is messing me up
total_payments_extended = total_payments.reindex(combined_df.index) #In dataframes, all columns must be the same length. 

combined_df['Total Payment'] = total_payments_extended['Total Payment'].values
combined_df['Unique Period'] = total_payments_extended['Unique Period'].values
 
with pd.ExcelWriter(export_file, engine='openpyxl') as writer:
    combined_df.to_excel(writer, index=False, sheet_name='Combined Amortization Tables')   
   
   
    
   
        
   

    